# Analysis of Partisan Facebook pages

In [10]:
import pandas
import pandas as pd
import numpy as np
import operator
import glob
from os.path import expanduser as ospath
from urlparse import urlparse
import os
import requests
from requests_toolbelt.utils import dump
import unicodedata
import facebook
from collections import Counter
import json


json_data1 = open('data/fb_pages_metrics.json').read()
partisan_data = json.loads(json_data1)
active_page_ids = []
dir_name = 'data/statuses'
json_pattern = os.path.join(dir_name,'*.csv')
file_list = glob.glob(json_pattern)

In [19]:
json_data2 = open('data/fb_mainstream_pages.json').read()
mainstream_data = json.loads(json_data2)

In [11]:
map_df = pandas.read_csv('data/fb_pages_mapping.csv')
active_ids = list(map_df['id'])
len(map_df)

487

In [20]:

def get_frame_data(fb_data):
    data = {}
    category_list = []
    id_list = []
    verification_list = []
    name_list = []
    fan_count_list = []
    website_list = []
    link_list = []
    start_type_list = []
    start_date_list = []
    for key,val in fb_data.items():
        id_list.append(val['id'])
        if 'category' in val.keys():
            category_list.append(val['category'])
        else:
            category_list.append(0)
        if 'verification_status' in val.keys():
            verification_list.append(val['verification_status'])
        else:
            verification_list.append(0)
        if 'name' in val.keys():
            name_list.append(val['name'])
        else:
            name_list.append(0)
        if 'fan_count' in val.keys():
            fan_count_list.append(val['fan_count'])
        else:
            fan_count_list.append(0)
        if 'website' in val.keys():
            website_list.append(val['website'])
        else:
            website_list.append(0)
        if 'link' in val.keys():
            link_list.append(val['link'])
        else:
            link_list.append(0)
        if 'start_info' in val.keys():
            if 'type' in val['start_info'].keys():
                start_type_list.append(val['start_info']['type'])
            else:
                start_type_list.append(0)
            if 'date' in val['start_info'].keys():
                if len(val['start_info']['date'].keys()) > 2:
                    start_date_list.append(str(val['start_info']['date']['day'])+"/"+str(val['start_info']['date']['month'])+"/"+str(val['start_info']['date']['day']))
                elif len(val['start_info']['date'].keys()) > 1:
                    start_date_list.append(str(val['start_info']['date']['month'])+"/"+str(val['start_info']['date']['year']))
                elif len(val['start_info']['date'].keys()) == 1:
                    start_date_list.append(str(val['start_info']['date']['year']))
                else:
                    start_date_list.append(0)
            else:
                start_date_list.append(0)
        else:
            start_type_list.append(0)
            start_date_list.append(0) 
    data['id'] = id_list
    data['name'] = name_list
    data['category'] = category_list
    data['fan_count'] = fan_count_list
    data['link'] = link_list
    data['website'] = website_list
    data['verification_status'] = verification_list
    data['start_type'] = start_type_list
    data['start_date'] = start_date_list
    return data

In [21]:
partisan_frame_data = get_frame_data(partisan_data)

In [25]:
mainstream_frame_data = get_frame_data(mainstream_data)

In [29]:
fb_df = pd.DataFrame(partisan_frame_data, columns=['name', 'id', 'category', 'verification_status', 'fan_count' , 'start_type', 'start_date','website', 'link' ])


In [30]:
mainstream_df = pd.DataFrame(mainstream_frame_data, columns=['name', 'id', 'category', 'verification_status', 'fan_count' , 'start_type', 'start_date','website', 'link' ])


In [35]:
filename = 'data/mainstream_pages.csv'
mainstream_df.to_csv(filename, index=False, encoding='utf-8')

In [8]:
# separate status.csv files into active and inactive files
active_file_list = []
inactive_file_list = []
for each_file in file_list:
    page_id = each_file.split('/')[2].split('_')[0]
    if page_id in list(fb_df['id']):
        active_file_list.append(each_file)
    else:
        inactive_file_list.append(each_file)

In [14]:
mean_loves_dict = {}
mean_sads_dict = {}
mean_wows_dict = {}
mean_hahas_dict = {}
mean_comments_dict = {}
mean_reactions_dict = {}
mean_shares_dict = {}
mean_likes_dict = {}
mean_angrys_dict = {}
for each_file in active_file_list:
    file_df = pandas.read_csv(each_file)
    page_id = each_file.split('/')[2].split('_')[0]
    mean_reactions_dict[page_id] = file_df['num_reactions'].mean()
    mean_comments_dict[page_id] = file_df['num_comments'].mean()
    mean_shares_dict[page_id] = file_df['num_shares'].mean()
    mean_likes_dict[page_id] = file_df['num_likes'].mean()
    mean_loves_dict[page_id] = file_df['num_loves'].mean()
    mean_wows_dict[page_id] = file_df['num_wows'].mean()
    mean_hahas_dict[page_id] = file_df['num_hahas'].mean()
    mean_sads_dict[page_id] = file_df['num_sads'].mean()
    mean_angrys_dict[page_id] = file_df['num_angrys'].mean()
   

In [154]:
engagement_data = {}
engagement_data['id'] = mean_reactions_dict.keys()
engagement_data['mean_reactions'] = mean_reactions_dict.values()
engagement_data['mean_comments'] = mean_reactions_dict.values()
engagement_data['mean_shares'] = mean_shares_dict.values()
engagement_data['mean_likes'] = mean_likes_dict.values()
engagement_data['mean_loves'] = mean_loves_dict.values()
engagement_data['mean_wows'] = mean_wows_dict.values()
engagement_data['mean_hahas'] = mean_hahas_dict.values()
engagement_data['mean_sads'] = mean_sads_dict.values()
engagement_data['mean_angrys'] = mean_angrys_dict.values()


In [155]:
eng_df = pd.DataFrame(engagement_data, columns=['id','mean_reactions','mean_comments','mean_shares','mean_likes','mean_loves','mean_wows','mean_hahas','mean_sads','mean_angrys'])

In [158]:
page_df = pd.merge(fb_df, eng_df, on='id')

In [ ]:
filename = 'data/Fb_Pages.csv'
page_df.to_csv(filename, index=False, encoding='utf-8')

In [3]:
Fb_page_df = pandas.read_csv('data/Fb_pages.csv')

In [18]:
len(fb_data)

485

In [ ]:
Fb_page_df.sort_values(['fan_count'], ascending=False)

In [20]:
count = 0
page_website_dict = {}
websites = []
for key, val in fb_data.items():
    if val.has_key('website') == False: 
        count += 1
    else:
        page_website_dict[key] = val['website'].split(" ")


In [22]:
# find intersection of partisan sites & fb sites
from partisan_sites_parser import *
len(sites)

668

In [23]:
fb_websites = []
for val in page_website_dict.values():
    for each_val in val:
        fb_websites.append(each_val)
        

In [24]:
for each_site in fb_websites:
    if each_site == '':
        fb_websites.remove(each_site)

In [25]:
len(set(fb_websites))

513

In [26]:
from urlparse import urlparse

fb_page_websites = []
fb_websites

for each_site in fb_websites:
    if 'http' in each_site:
        if 'www' in urlparse(each_site).netloc:
            fb_page_websites.append(urlparse(each_site).netloc[4:])
        #print urlparse(each_site).netloc
        else:
            if urlparse(each_site).netloc != '':
                fb_page_websites.append(urlparse(each_site).netloc)
            else:
                fb_page_websites.append(urlparse(each_site).path)
        #pass
    elif 'www' in each_site:
        fb_page_websites.append(each_site[4:])
    else:
        fb_page_websites.append(each_site)

In [27]:
len(set(fb_page_websites ))

506

In [22]:
for each_site in fb_page_websites:
    if 'rightpundits' in each_site or 'www' in each_site or each_site == ":" or each_site == " ":
        fb_page_websites.remove(each_site)
   

In [23]:
xlist = ['rightpundits.com','military.rightpundits.com','food.rightpundits.com', 'law.rightpundits.com','celebrity.rightpundits.com','sports.rightpundits.com','lauraingraham.com','twitter.com/ingrahamangle']
for each_site in xlist:
    fb_page_websites.append(each_site)


In [24]:
#total fb websites
len(set(fb_page_websites)) 

510

In [25]:
fb_page_websites = [x.lower() for x in fb_page_websites]
len(fb_page_websites)

518

In [26]:
Counter(fb_page_websites).most_common(10)

[(u'twitter.com', 5),
 (u'frontpagemag.com', 3),
 (u'cscmediagroupus.com', 2),
 (u'thetruthdivision.com', 2),
 (u'foxnews.com', 2),
 (u'twitter.com/hudsoninstitute', 1),
 (u'deadstate.org', 1),
 (u'redflagnews.com', 1),
 (u'uspoliticsinfo.com', 1),
 (u'ahtribune.com', 1)]

In [27]:
# intersection of partisan sites and fb sites
from partisan_sites_parser import *
partisan_websites  = sites
common_partisan_sites = set(partisan_websites).intersection(set(fb_page_websites))

In [28]:
len(common_partisan_sites)

372

In [29]:
fb_left = []
fb_right = []
fb_undecided = []
for each_site in list(set(fb_page_websites)):
    if each_site in left:
        fb_left.append(each_site)
    elif each_site in right:
        fb_right.append(each_site)
    else:
        fb_undecided.append(each_site)

In [54]:
page_names = list(Fb_page_df['name'])

In [62]:
fb_website_list = list(Fb_page_df['website'])

In [108]:
for each_site in fb_website_list:
    for each_val in fb_left:
        if each_val in each_site:
            category_dict[each_site] = 'left'
    for each_val in fb_right:
        if each_val in each_site:
            category_dict[each_site] = 'right'
    for each_val in fb_undecided:
        if each_val in each_site:
            category_dict[each_site] = 'uncategorized'
        

In [110]:
len(category_dict)

459

In [111]:
category_data = {}
category_data['website'] = category_dict.keys()
category_data['political_category'] = category_dict.values()

In [112]:
category_df = pd.DataFrame(category_data, columns=['website', 'political_category'])

In [113]:
new_page_df = pd.merge(Fb_page_df, category_df, on='website')

In [ ]:
filename = 'data/fb_Pages_2.csv'
new_page_df.to_csv(filename, index=False, encoding='utf-8')

In [4]:
new_pg_df = pandas.read_csv('data/Fb_pages_2.csv')

In [64]:
len(new_pg_df[new_pg_df['political_category'] =='left'])

112

In [65]:
len(new_pg_df[new_pg_df['political_category'] =='right'])

236

In [50]:
list(new_pg_df[new_pg_df['political_category']=='left'].sort_values('fan_count', ascending=False).head(10)['id'])

[354522044588660,
 346937065399354,
 114517875225866,
 160389977329803,
 610045389164725,
 273864989376427,
 210277954204,
 200137333331078,
 1672814509645693,
 7642602143]

In [51]:
list(new_pg_df[new_pg_df['political_category']=='right'].sort_values('fan_count', ascending=False).head(10)['id'])

[184859785026060,
 179035672287016,
 123624513983,
 182919686769,
 133961323610549,
 389658314427637,
 95475020353,
 201956993160690,
 69813760388,
 112723252096438]

In [47]:
new_pg_df.sort_values('fan_count', ascending=False).head(10)

,name,id,category,verification_status,fan_count,start_type,start_date,website,link,mean_reactions,mean_comments,mean_shares,mean_likes,mean_loves,mean_wows,mean_hahas,mean_sads,mean_angrys,political_category
342,Fox News,15704546335,Media/News Company,blue_verified,15614839,Founded,7/10/7,"http://foxnews.com/, http://insider.foxnews.com",https://www.facebook.com/FoxNews/,19609.953890,19609.953890,5534.918118,16838.592555,929.569362,208.332993,460.854554,317.919494,854.313377,uncategorized
347,NowThis,341163402640457,News & Media Website,blue_verified,11776185,Founded,2012,http://weibo.com/NowThis,https://www.facebook.com/NowThisNews/,16220.240654,16220.240654,11827.127377,13510.533194,1281.515952,338.161537,278.946906,463.802046,346.765147,uncategorized
286,Upworthy,354522044588660,Media/News Company,blue_verified,11285519,Launched,3/2012,http://www.upworthy.com,https://www.facebook.com/Upworthy/,10321.425824,10321.425824,4287.502177,9285.294019,719.738278,90.462549,44.765641,135.773988,44.680723,left
455,ForAmerica,114364638621903,Nonprofit Organization,blue_verified,7919038,Launched,1/9/1,www.ForAmerica.org,https://www.facebook.com/ForAmerica/,39389.349192,39389.349192,12294.727630,38043.266696,301.254038,87.677870,319.701440,177.975557,459.401571,uncategorized
100,Occupy Democrats,346937065399354,Political Organization,blue_verified,6718910,Founded,21/9/21,www.occupydemocrats.com,https://www.facebook.com/OccupyDemocrats/,29131.514028,29131.514028,22710.509483,24022.561285,1215.064890,433.397100,1712.826411,424.911129,1322.481505,left
325,Faith Family America,184859785026060,Publisher,not_verified,5963753,Unspecified,0,http://www.faithfamilyamerica.com/,https://www.facebook.com/faithfamilyamerica/,3639.677304,3639.677304,2204.573683,3413.712095,126.166423,20.235740,17.500022,24.325223,37.679165,right
360,American News,179035672287016,Other,gray_verified,5565048,Unspecified,0,http://americannews.com,https://www.facebook.com/ThePatriotReview/,14886.489606,14886.489606,4494.879024,14314.091844,184.432525,31.990197,90.172704,51.829521,212.831259,right
44,Western Journalism,123624513983,News & Media Website,blue_verified,5227316,Launched,25/9/25,http://www.westernjournalism.com,https://www.facebook.com/WesternJournalism/,5291.808890,5291.808890,1785.777467,5010.206077,75.476669,35.089446,54.567603,22.134214,94.298679,right
199,NRA Institute for Legislative Action,22561081832,Nonprofit Organization,blue_verified,5118919,Founded,1871,http://www.NRAILA.org,https://www.facebook.com/NationalRifleAssociat...,9868.035613,9868.035613,2960.681101,9421.320113,100.805342,25.624039,58.378794,16.460138,245.390530,uncategorized
145,The Other 98%,114517875225866,Broadcasting & Media Production Company,blue_verified,5059627,Born,15/4/15,http://other98.com,https://www.facebook.com/TheOther98/,16855.395029,16855.395029,12862.244034,13714.705048,677.045420,220.759155,1070.292313,314.278346,858.225558,left


In [69]:
Hillary_list = []
for each_page in list(new_pg_df['name']):
    if 'Hillary' in each_page or 'Clinton' in each_page:
        Hillary_list.append(each_page)

In [70]:
Trump_list = []
for each_page in list(new_pg_df['name']):
    if 'Trump' in each_page or 'Donald' in each_page:
        Trump_list.append(each_page)

In [ ]:
Trump_list = []
for each_page in list(new_pg_df['name']):
    if 'America' in each_page or 'American' in each_page:
        print each_page

In [ ]:
Hillary_list

In [ ]:
Trump_list

In [ ]:
for each_page in list(new_pg_df['name']):
    if 'Right' in each_page or 'Right' in each_page or 'Conservative' in each_page or 'conservative' in each_page or 'republican' in each_page or 'Republican' in each_page:
        print each_page
        

In [ ]:
new_pg_df[new_pg_df['name'].isin(Hillary_list)].sort_values(['fan_count'], ascending=False)

In [ ]:
new_pg_df[new_pg_df['name'].isin(Trump_list)].sort_values(['fan_count'], ascending=False)

In [ ]:
for each_page in list(new_pg_df['name']):
    if 'left' in each_page or 'Left' in each_page or 'Liberal' in each_page or 'liberal' in each_page or 'democrat' in each_page or 'Democrat' in each_page:
        print each_page

In [137]:
domain_dict = {}

In [228]:
for each_file in file_list:
    each_id = each_file.split("/")[2].split("_")[0]
    if int(each_id) in active_ids:
        page_name = list(map_df[map_df['id'] == int(each_id)]['page_name'])[0]
        #print page_name
        domain_dict[page_name] = {}
        df = pandas.read_csv(each_file)
        domain_list = []
        link_df = df['status_link']
        link_list = list(link_df)
        c = 0
        for each_link in link_list:
            try:
                parsed_url = urlparse(each_link)
                domain_list.append(parsed_url.netloc)
            except:
                #print parsed_url.netloc
                domain_list.append(parsed_url.netloc)
        domain_dict[page_name] = dict(Counter(domain_list))

In [167]:
from partisan_sites_parser import *
partisan_websites = sites

In [244]:
internal_ref_dict = {}
for key, val in domain_dict.items():
    for each_key, each_val in val.items():
        if 'www' in each_key:
            url = each_key[4:]
        else:
            url = each_key
        if url in partisan_websites:
            if internal_ref_dict.has_key(key) == False:
                internal_ref_dict[key] = each_val
            else:
                internal_ref_dict[key] += each_val
       

In [211]:
fb_data = pandas.read_csv('data/fb_pages_mapping.csv')

In [245]:
for key, val in internal_ref_dict.items():
    perc = (val/float(fb_data[fb_data['page_name'] == key]['status_count']))*100
    internal_perc_dict[key] = perc

In [251]:
np.mean(internal_perc_dict.values())

58.918399570635287

In [196]:
for x in domain_dict['Proud Liberal'].keys():
    if x in partisan_websites:
        print domain_dict['Proud Liberal'][x]

1
3
4
988


In [198]:
domain_dict['Proud Liberal']

{'10wealthymen.newsvine.com': 1,
 'bluedotdaily.com': 4,
 'kai0.newsvine.com': 1,
 'krishna109.newsvine.com': 1,
 'media.giphy.com': 1,
 'mic.com': 1,
 'mobile.nytimes.com': 2,
 'rawprogressive.com': 1,
 'thehill.com': 3,
 'theproudliberal.org': 988,
 'thinkprogress.org': 3,
 'www.bluedotdaily.com': 133,
 'www.cbsnews.com': 1,
 'www.cnbc.com': 1,
 'www.cnn.com': 8,
 'www.dailykos.com': 1,
 'www.esquire.com': 1,
 'www.facebook.com': 13,
 'www.huffingtonpost.com': 3,
 'www.ifyouonlynews.com': 1,
 'www.msnbc.com': 1,
 'www.nydailynews.com': 1,
 'www.rawprogressive.com': 47,
 'www.theguardian.com': 1,
 'www.washingtonpost.com': 3,
 'www.yahoo.com': 2}